In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder as le

In [ ]:
comments_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")
valid_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")
sub_df=pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
prev_train_df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
prev_train_df

In [ ]:
prev_train_df["severe_toxic"] = prev_train_df["severe_toxic"] *3
prev_train_df["threat"] = prev_train_df["threat"] *2.5
prev_train_df["identity_hate"] = prev_train_df["identity_hate"]*3

prev_train_df["total_toxic"] = prev_train_df[["toxic", "severe_toxic", "obscene", "threat", "insult",
                                              "identity_hate"]].sum(axis = 1).astype(np.int64)

In [ ]:
train_df = prev_train_df[["comment_text", "total_toxic"]]
train_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tfvec = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (3,5))
tfv = tfvec.fit_transform(train_df["comment_text"])

In [ ]:
X=tfv[1:25001]
Y=train_df.loc[1:25000,['total_toxic']]

In [ ]:
from sklearn.linear_model import LinearRegression as lr

reg = lr().fit(X,Y)
tfv_comments = tfvec.transform(comments_df["text"])
pred = reg.predict(tfv_comments)
a=pred[:1000]
reg.score(X,Y) 

import xgboost as xg
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 123)
xgb_r=xgb_r.fit(X, Y)
tfv_comments = tfvec.transform(comments_df["text"])
# Predict the model
pred = xgb_r.predict(tfv_comments)
a=pred[:1000]
xgb_r.score(X,Y)


In [ ]:
sub = pd.DataFrame()
sub["comment_id"] = comments_df["comment_id"]
sub["score"] = pred
sub.to_csv('submission.csv',index=False)
sub